In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


### CREACIÓN DE DATASETS

In [ ]:
df_orders = pd.read_excel('./datasets/df_orders.xlsx')
df_vehicle = pd.read_excel('./datasets/df_vehicle.xlsx')
df_location = pd.read_excel('./datasets/df_location.xlsx')
df_historic_orders = pd.read_excel('./datasets/df_historic_order_demand.xlsx')
df_distance_min = pd.read_excel('./datasets/df_distance_min.xlsx')
df_distance_km = pd.read_excel('./datasets/df_distance_km.xlsx')

### ANÁLISIS DESCRIPTIVO

Lo primero determinamos cual es el objetivo a la hora de horientar nuestro modelo. Para ello determinaremos cuáles pueden ser nuestras tablas en las que apoyar todo nuestro trabajo. Para ello vemos que la tabla de orders e historic_order_demand son las más relevantes para dicho análisis.

Decidimos separar la columna de mes_año por dos columnas. Siendo estas dos mes y año.

In [ ]:
df_orders[["mes", "año"]] = df_orders["mes_anio"].str.split("-", expand=True).astype(int)
df_orders.drop(columns=["mes_anio"], inplace=True)

df_historic_orders[["mes", "año"]] = df_historic_orders["mes_anio"].str.split("-", expand=True).astype(int)
df_historic_orders.drop(columns=["mes_anio"], inplace=True)

In [ ]:
df_orders.head(5)

In [ ]:
df_historic_orders.head(5)

In [ ]:
print("Dimensiones de las distancias en Kilómetros:", df_distance_km.shape)
print("Dimensiones de las distancias en Minutos:", df_distance_min.shape)

In [ ]:
# Resumen estadístico de las distancias
print("\nResumen de las distancias en Kilómetros:")
df_distance_km.describe()

In [ ]:

print("\nResumen de las distancias en Minutos:")
df_distance_min.describe()

Creamos nuevos datasets para ver la disntacia tanto en kilometros como del almacén de los diferentes clientes

In [ ]:
df_distacia_almacen_km = df_distance_km[["Almacén"]].copy()
df_distacia_almacen_km["cliente"] = df_distance_km.index + 1

In [ ]:
df_distacia_almacen_min = df_distance_min[["Almacén"]].copy()
df_distacia_almacen_min["cliente"] = df_distance_min.index + 1

Estandarizamos la columna de clientes en los diferentes datasets

In [ ]:
def estandarizar_cliente(cliente):
    if isinstance(cliente, int):
        return f'cliente_{cliente}'
    else:
        return str(cliente).strip().lower()


datasets = [df_historic_orders, df_orders, df_distacia_almacen_km, df_distacia_almacen_min]
for df in datasets:
    df["cliente"] = df["cliente"].apply(estandarizar_cliente)


In [ ]:
df_historic_orders.head(5)

In [ ]:
df_distacia_almacen_min.head(5)

#### Relación entre la Distancia del Almacén y la Demanda de Pedidos

In [ ]:
total_demand_per_client = df_historic_orders.groupby('cliente')
total_demand_per_client

In [ ]:
df_distance_demand = pd.merge(df_distacia_almacen_km, total_demand_per_client, on='cliente')

fig = px.scatter(df_distance_demand, x='Almacén', y='order_demand', text='cliente', title='Relación entre la Distancia del Almacén y la Demanda de Pedidos')
fig.update_layout(xaxis_title='Distancia al Almacén (km)', yaxis_title='Demanda Total de Pedidos (kg)')
fig.show()

#### Relación entre el tiempo Almacén-Cliente y la Demanda de Pedidos

In [ ]:
df_time_demand = pd.merge(df_distacia_almacen_min, total_demand_per_client, on='cliente')

fig = px.scatter(df_time_demand, x='Almacén', y='order_demand', text='cliente', title='Relación entre la tiempo Almacén-Cliente y la Demanda de Pedidos')
fig.update_layout(xaxis_title='Tiempo desde el almacén(minutos)', yaxis_title='Demanda Total de Pedidos (kg)')
fig.show()

#### Demanda mensual total por mes

In [ ]:
# Calcular la demanda mensual total
df_monthly_demand = df_historic_orders.groupby(['año', 'mes']).agg({'order_demand': 'sum'}).reset_index()

# Crear el gráfico de demanda mensual total
fig = px.line(df_monthly_demand, x='mes', y='order_demand', color='año', markers=True, title='Demanda Mensual Total')
fig.update_layout(xaxis_title='Mes', yaxis_title='Demanda Total (kg)')
fig.show()

#### Distribución de la demanda de pedidos en el último mes.

In [ ]:
fig1 = px.histogram(df_orders, x='order_demand', nbins=10, title='Distribución de la Demanda de Pedidos en el Último Mes')
fig1.update_layout(xaxis_title='Demanda de Pedidos (kg)', yaxis_title='Frecuencia')
fig1.show()

#### Tendencia de la demanda histórica de pedidos.

In [ ]:
fig2 = px.line(df_historic_orders, x='año', y='order_demand', color='cliente', markers=True, title='Tendencia de la Demanda Histórica de Pedidos')
fig2.update_layout(xaxis_title='Año', yaxis_title='Demanda de Pedidos (kg)')
fig2.show()


#### Capacidad y Costo por km de los Vehículos

In [ ]:
fig3 = px.scatter(df_vehicle, x='capacidad_kg', y='costo_km', size='autonomia_km', title='Capacidad y Costo por km de los Vehículos')
fig3.update_layout(xaxis_title='Capacidad (kg)', yaxis_title='Costo por km')
fig3.show()

#### Localización geográfica de los clientes

In [ ]:
fig6 = px.scatter(df_location, x='Longitud', y='Latitud', text='Cliente', title='Localización Geográfica de los Clientes')
fig6.update_traces(textposition='top center')
fig6.update_layout(xaxis_title='Longitud', yaxis_title='Latitud')
fig6.show()